In [15]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col,sum,when


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

In [2]:
Salesperson = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Salesperson") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

Customer = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Customer") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

Sales = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Sales") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [16]:
Sales = Sales.withColumnRenamed("customer_id","c_id")
Customer = Customer.withColumnRenamed("salesperson_id","s_id")


total_sales_df = Sales\
    .join(Customer,on=Sales.c_id == Customer.customer_id,how="inner")\
    .join(Salesperson,on=Customer.s_id==Salesperson.salesperson_id,how="inner")\
    .select("salesperson_id","price")\
    .groupBy(col("salesperson_id").alias("t_s_id"))\
    .agg(sum("price").alias("total"))

total_sales_df\
    .join(Salesperson,on=total_sales_df.t_s_id==Salesperson.salesperson_id,how="right")\
    .withColumn("total_sales",when(col("total").isNull(),0).otherwise(col("total")))\
    .select("salesperson_id","name","total_sales")\
    .show()



+--------------+-----+-----------+
|salesperson_id| name|total_sales|
+--------------+-----+-----------+
|             1|Alice|    1246.00|
|             2|  Bob|    1844.00|
|             3|Jerry|       0.00|
+--------------+-----+-----------+

